In [1]:
#this script checks missing and present data for participants
#this is currently set up for the AOL study Qualtrics measures
#
#TO USE:
#download .csv files with labels and extract the zipped file
#put the unzipped file in the data_dir directory
#update the participants you need to check
#save the file to whatever directory works best for you
# 
# BP 7/30/20

#SET UP#

#calling libraries
import pandas as pd
import xlwt
from xlwt import Workbook

#subjects to check
subs = ['131001', '132001', '132002', '123456']

#directory where data is stored. Must have double \\ not just a single \
data_dir = 'C:\\Users\\Becca\Dropbox (UFL)\\Lab Studies\\AOL\\data back up checks\\data\\'
#directory where you want to save the check output files. Must have double \\ not just a single \
save_dir = 'C:\\Users\\Becca\Dropbox (UFL)\\Lab Studies\\AOL\\data back up checks\\data\\'

#must make sure file names match the data you want to use. Must specify csv
#sv_icf_file = 
#sv_id_column = 
#audio_icf_file = 
#audio_id_column = 
sigt_file="AOL_ SIGT Player preference_July 17, 2020_08.19.csv"
face_a_file="AOL_FACE TASK_A._July 17, 2020_08.19.csv"
face_b_file="AOL_FACE TASK_B._July 17, 2020_08.19.csv"
video_a_file=
video_b_file=
picture_file = "AOL Picture Test_July 17, 2020_08.16.csv"
vocab_file=
fnt_file=
temporal_gains_file="AOL Temporal Discounting Gains_July 17, 2020_08.17.csv"
temporal_losses_file="AOL Temporal Discounting Losses_July 17, 2020_08.17.csv"

redcap_filename="AgingOnLine_DATA_2020-08-02_2326.csv"

#column names where subID is located. this should not change.
sigt_id_column= "Q65"
face_a_id_column="ID"
face_b_id_column="ID"
video_a_id_column=
video_b_id_column=
picture_id_column = "Q36"
vocab_id_column=
fnt_id_column=
temporal_gains_id_column="What is your participant ID?"
temporal_losses_id_column="What is your participant ID?"

#file names to save output. Update with date of most recent data (match the datafile)
save_name_present = 'qualtrics_present_files_8_2_20.xls'
save_name_missing = 'qualtrics_missing_files_8_2_20.xls'

#opening 2 workbooks to write results
wb1 = Workbook()
wb2 = Workbook()

SyntaxError: invalid syntax (<ipython-input-1-02fd17b41a1e>, line 30)

In [1]:
#1. SV ICF

#CANNOT CHECK -- no subIDs

In [113]:
#2. Audio ICF

#CANNOT CHECK -- no subIDs

In [119]:
#3. SIGT player preference

#setting the column to read and the file to read from
id_column = sigt_id_column
filename = sigt_file

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('SIGT present')
sheet2 = wb2.add_sheet('SIGT missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#creating a list of 1 header name. Only this 1 column will be read in
col_list = [id_column]

#reading in the qualtrics file with only the header specified
df = pd.read_csv(r'' + data_dir + filename,  usecols=col_list)

#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list2 = ["record_id", "v5_sigt_alreadydone"]
df2 = pd.read_csv(r'' + data_dir + redcap_filename,  usecols=col_list2)

#turning the imported excel column into a list so we can manipulate the data
data_list = df[id_column].tolist()
data_list2 = df2["v5_sigt_alreadydone"].tolist()

for x in subs:
    if x in data_list:
        place_counter_complete=place_counter_complete+1 #updating the present place counter so we will write to a new cell
        sheet1.write(place_counter_complete, 0, x) #write the subID (x) into the correct cell in the complete workbook
    if x not in data_list:
        place_counter_missing=place_counter_missing+1 #updating the missing place counter so we will write to a new cell
        sheet2.write(place_counter_missing, 0, x) #write the subID (x) into the correct cell in the missing workbook
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

In [ ]:
#4. Pre assessment

#has different PHIT IDs, so it has its own check code
#see assessment_check.ipynb to run

In [121]:
#5. FACE task A

id_column = face_a_id_column
filename = face_a_file

sheet1 = wb1.add_sheet('FACE task A present')
sheet2 = wb2.add_sheet('FACE task A missing')
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

col_list = [id_column]
df = pd.read_csv(r'' + data_dir + filename,  usecols=col_list)
data_list = df[id_column].tolist()

for x in subs:
    if (int(x[5]) % 2) != 0:  #checking if last digit of subID is odd
        if x in data_list:
            place_counter_complete=place_counter_complete+1
            sheet1.write(place_counter_complete, 0, x)
        if x not in data_list:
            place_counter_missing=place_counter_missing+1
            sheet2.write(place_counter_missing, 0, x)
        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

In [ ]:
#6. FACE task B

id_column = face_b_id_column
filename = face_b_file

sheet1 = wb1.add_sheet('FACE task B present')
sheet2 = wb2.add_sheet('FACE task B missing')
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

col_list = [id_column]
df = pd.read_csv(r'' + data_dir + filename,  usecols=col_list)
data_list = df[id_column].tolist()

#print(df)
#print(data_list)
#print(subs)

for x in subs:
    if (int(x[5]) % 2) == 0:  #checking if last digit of subID is even
        if x in data_list:
            place_counter_complete=place_counter_complete+1
            sheet1.write(place_counter_complete, 0, x)
        if x not in data_list:
            place_counter_missing=place_counter_missing+1
            sheet2.write(place_counter_missing, 0, x)
        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

In [ ]:
#7. Lie video task A

id_column = video_a_id_column
filename = video_a_file

sheet1 = wb1.add_sheet('Video task A present')
sheet2 = wb2.add_sheet('Video task A missing')
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

col_list = [id_column]
df = pd.read_csv(r'' + data_dir + filename,  usecols=col_list)
data_list = df[id_column].tolist()

#print(df)
#print(data_list)
#print(subs)

for x in subs:
    if (int(x[5]) % 2) != 0:  #checking if last digit of subID is odd
        if x in data_list:
            place_counter_complete=place_counter_complete+1
            sheet1.write(place_counter_complete, 0, x)
        if x not in data_list:
            place_counter_missing=place_counter_missing+1
            sheet2.write(place_counter_missing, 0, x)
        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

In [ ]:
#8. Lie video task B

id_column = video_b_id_column
filename = video_b_file

sheet1 = wb1.add_sheet('Video task A present')
sheet2 = wb2.add_sheet('Video task A missing')
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

col_list = [id_column]
df = pd.read_csv(r'' + data_dir + filename,  usecols=col_list)
data_list = df[id_column].tolist()

#print(df)
#print(data_list)
#print(subs)

for x in subs:
    if (int(x[5]) % 2) == 0:  #checking if last digit of subID is even
        if x in data_list:
            place_counter_complete=place_counter_complete+1
            sheet1.write(place_counter_complete, 0, x)
        if x not in data_list:
            place_counter_missing=place_counter_missing+1
            sheet2.write(place_counter_missing, 0, x)
        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

In [ ]:
#9. Picture task

id_column = picture_id_column
filename = picture_file

sheet1 = wb1.add_sheet('Picture task present')
sheet2 = wb2.add_sheet('Picture task missing')
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

col_list = [id_column]
df = pd.read_csv(r'' + data_dir + filename,  usecols=col_list)
data_list = df[id_column].tolist()

#print(df)
#print(data_list)
#print(subs)

for x in subs:
    if x in data_list:
        place_counter_complete=place_counter_complete+1
        sheet1.write(place_counter_complete, 0, x)
    if x not in data_list:
        place_counter_missing=place_counter_missing+1
        sheet2.write(place_counter_missing, 0, x)
        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

In [ ]:
#10. Shipley vocab

id_column = vocab_id_column
filename = vocab_file

sheet1 = wb1.add_sheet('Vocab task present')
sheet2 = wb2.add_sheet('Vocab task missing')
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

col_list = [id_column]
df = pd.read_csv(r'' + data_dir + filename,  usecols=col_list)
data_list = df[id_column].tolist()

#print(df)
#print(data_list)
#print(subs)

for x in subs:
    if x in data_list:
        place_counter_complete=place_counter_complete+1
        sheet1.write(place_counter_complete, 0, x)
    if x not in data_list:
        place_counter_missing=place_counter_missing+1
        sheet2.write(place_counter_missing, 0, x)
        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

In [ ]:
#11. FNT

id_column = fnt_id_column
filename = fnt_file

sheet1 = wb1.add_sheet('FNT present')
sheet2 = wb2.add_sheet('FNT missing')
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

col_list = [id_column]
df = pd.read_csv(r'' + data_dir + filename,  usecols=col_list)
data_list = df[id_column].tolist()

#print(df)
#print(data_list)
#print(subs)

for x in subs:
    if x in data_list:
        place_counter_complete=place_counter_complete+1
        sheet1.write(place_counter_complete, 0, x)
    if x not in data_list:
        place_counter_missing=place_counter_missing+1
        sheet2.write(place_counter_missing, 0, x)
        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

In [ ]:
#12. Temporal discounting gains

id_column = temporal_gains_id_column
filename = temporal_gains_file

sheet1 = wb1.add_sheet('Temporal gains present')
sheet2 = wb2.add_sheet('Temporal gains missing')
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

col_list = [id_column]
df = pd.read_csv(r'' + data_dir + filename,  usecols=col_list)
data_list = df[id_column].tolist()

#print(df)
#print(data_list)
#print(subs)

for x in subs:
    if (int(x[5]) % 2) != 0:  #checking if last digit of subID is odd
        if x in data_list:
            place_counter_complete=place_counter_complete+1
            sheet1.write(place_counter_complete, 0, x)
        if x not in data_list:
            place_counter_missing=place_counter_missing+1
            sheet2.write(place_counter_missing, 0, x)
        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

In [ ]:
#13. Temporal discounting losses

id_column = temporal_losses_id_column
filename = temporal_losses_file

sheet1 = wb1.add_sheet('Temporal losses present')
sheet2 = wb2.add_sheet('Temporal losses missing')
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

col_list = [id_column]
df = pd.read_csv(r'' + data_dir + filename,  usecols=col_list)
data_list = df[id_column].tolist()

#print(df)
#print(data_list)
#print(subs)

for x in subs:
    if (int(x[5]) % 2) == 0:  #checking if last digit of subID is even
        if x in data_list:
            place_counter_complete=place_counter_complete+1
            sheet1.write(place_counter_complete, 0, x)
        if x not in data_list:
            place_counter_missing=place_counter_missing+1
            sheet2.write(place_counter_missing, 0, x)
        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

In [ ]:
#14. Assessment 1

#has different PHIT IDs, so it has its own check code
#see assessment_check.ipynb to run

In [ ]:
#15. Assessment 2

#has different PHIT IDs, so it has its own check code
#see assessment_check.ipynb to run

In [ ]:
#16. Assessment 3

#has different PHIT IDs, so it has its own check code
#see assessment_check.ipynb to run

In [ ]:
#17. Assessment 4

#has different PHIT IDs, so it has its own check code
#see assessment_check.ipynb to run

In [ ]:
#18. Assessment 5

#has different PHIT IDs, so it has its own check code
#see assessment_check.ipynb to run

In [ ]:
#19. Assessment 6

#has different PHIT IDs, so it has its own check code
#see assessment_check.ipynb to run